In [7]:
library(rgdal)
library(spdep)
library(tidyverse)

In [14]:
install.packages("geojsonio")

Installing package into 'C:/Users/wozni/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
also installing the dependencies 'geometries', 'jsonify', 'rapidjsonr', 'sfheaders', 'protolite', 'geojsonsf', 'geojson'




  There is a binary version available but the source version is later:
          binary source needs_compilation
sfheaders  0.3.0  0.4.0              TRUE

  Binaries will be installed
package 'geometries' successfully unpacked and MD5 sums checked
package 'jsonify' successfully unpacked and MD5 sums checked
package 'rapidjsonr' successfully unpacked and MD5 sums checked
package 'sfheaders' successfully unpacked and MD5 sums checked
package 'protolite' successfully unpacked and MD5 sums checked
package 'geojsonsf' successfully unpacked and MD5 sums checked
package 'geojson' successfully unpacked and MD5 sums checked
package 'geojsonio' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\wozni\AppData\Local\Temp\RtmpOgbMMm\downloaded_packages


# Warszawa

In [40]:
df <- readOGR("../datasets/preprocessed_data/df_warszawa.shp")
df <- spTransform(df, CRS("+proj=longlat +ellps=GRS80 +no_defs"))
df.cont.nb <- poly2nb(as(df, "SpatialPolygons"), queen=TRUE)
df.cont.listw <- nb2listw(df.cont.nb, style="W")

OGR data source with driver: ESRI Shapefile 
Source: "D:\spatial_econometric_project\datasets\preprocessed_data\df_warszawa.shp", layer: "df_warszawa"
with 601 features
It has 31 fields
Integer64 fields read as strings:  grid_index 


## OLS

In [ ]:
model.lm <- lm(y~x1+x2+x3+x4, data=sub)
summary(model.lm)

# Kraków

In [42]:
krakow <- readOGR("../datasets/preprocessed_data/df_krakow.shp")
krakow <- spTransform(krakow, CRS("+proj=longlat +ellps=GRS80 +no_defs"))
krakow.cont.nb <- poly2nb(as(krakow, "SpatialPolygons"), queen=TRUE)
krakow.cont.listw <- nb2listw(krakow.cont.nb, style="W")

OGR data source with driver: ESRI Shapefile 
Source: "D:\spatial_econometric_project\datasets\preprocessed_data\df_krakow.shp", layer: "df_krakow"
with 396 features
It has 31 fields
Integer64 fields read as strings:  grid_index 


# Poznań

In [43]:
poznan <- readOGR("../datasets/preprocessed_data/df_poznan.shp")
poznan <- spTransform(poznan, CRS("+proj=longlat +ellps=GRS80 +no_defs"))
poznan.cont.nb <- poly2nb(as(warszawa, "SpatialPolygons"), queen=TRUE)
poznan.cont.listw <- nb2listw(poznan.cont.nb, style="W")

OGR data source with driver: ESRI Shapefile 
Source: "D:\spatial_econometric_project\datasets\preprocessed_data\df_poznan.shp", layer: "df_poznan"
with 324 features
It has 31 fields
Integer64 fields read as strings:  grid_index 
